In [1]:
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector

import requests
import pandas as pd
import time
from tqdm import tqdm

In [2]:
def get_tables(html_table, tag=['th', 'tr', 'td']):
    table_value = []


    # for my_table in tables:

    # You can find children with multiple tags by passing a list of strings
    rows = html_table.findChildren(tag)

    for row in rows:
        cells = row.findChildren(tag)
        for cell in cells:
            value = cell.string
            if value:
                table_value.append(value.strip())
                # print("The value in this cell is %s" % value)
            else:
                table_value.append("None")
    return dict(zip(table_value[::2], table_value[1::2]))

In [3]:
def parse_page(url):
    
    # get the html file
    page = requests.get(url)
    
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')

        # get all the tables
        tables = soup.findChildren('table')

        # only the first 4 tables are useful
        tables = tables[:4]


        # UPPER LEFT TABLE
        table_ul = get_tables(tables[0])

        # UPPER RIGHT TABLE
        rows_html = tables[1].findAll("span", {"class": lambda x: x and x.startswith("rating")})
        rows = [x.get_text() for x in rows_html]
        table_ur = dict(zip(rows[::2], rows[1::2]))

#         # BUTTOM TABLES

#         table_b = {}
#         for t in tables[2:]:
#             l = get_tables(t, ['td', 'p', 'tr'])
#             table_b.update(l)


        all_items = {**table_ul, **table_ur}
    
    else:
        print(page.status_code)


    return all_items

In [5]:
all_boards = []

for page in tqdm(range(100)):
    base_url = "https://thegoodride.com/snowboard-reviews/?mens={}".format(page)

    parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
    resp = requests.get(base_url, headers={"content-type":"text"})
    
    if resp.status_code == 200:
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        soup = BeautifulSoup(resp.content, parser, from_encoding=encoding)

        rows_html = soup.findAll("div", {"class": "board-reviews animate"})


        for board in rows_html:
            board_name = board.select('h4')[0].text.strip()
            review_url = board.select('a', href=True)[0]['href']
            all_boards.append([board_name, review_url])
    else:
        print("ERROR {}".format(resp.status_code))
    
    time.sleep(1)

100%|██████████| 100/100 [04:40<00:00,  2.81s/it]


In [8]:
df_url = pd.DataFrame(all_boards, columns=['board_name', 'url'])
df_url.shape

(4000, 2)

In [9]:
df_url.to_csv('../data/100_pages.csv', index=False)

In [7]:
# TODO: get date from url

In [ ]:
rating_list = []
for url in tqdm(df_url['url']):
    d = parse_page(url)
    assert len(d) == 18
    rating_list.append(d)
    time.sleep(1)

  2%|▏         | 71/4000 [03:09<1:34:09,  1.44s/it]

In [ ]:
df_rating = pd.DataFrame(rating_list)

In [ ]:
df_final = pd.concat([df_url, df_rating], axis=1)